In [10]:
import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from plotly.subplots import make_subplots



In [25]:
# =========================
# PARÂMETROS
# =========================
ticker = "PETR4.SA" # código do ativo no yahoo
start_date = "1998-01-01"
end_date = None


In [26]:
df = yf.download(
    ticker,
    start=start_date,
    end=end_date,
    progress=False,
    auto_adjust=True,
    back_adjust=True,
    multi_level_index=False,
    
)

df = df.dropna()
df['weekday'] = df.index.weekday
df.dropna(inplace=True)
df['ret'] = df['Close'].pct_change()
df.tail()



,Close,High,Low,Open,Volume,weekday,ret
Date,,,,,,,
2025-12-10,31.940001,31.990000,31.540001,31.879999,22123700,2,0.002511
2025-12-11,31.410000,31.900000,31.250000,31.879999,64927900,3,-0.016594
2025-12-12,31.590000,31.719999,31.340000,31.410000,39659900,4,0.005731
2025-12-15,31.700001,31.900000,31.590000,31.680000,31898900,0,0.003482
2025-12-16,30.740000,0.000000,0.000000,0.000000,0,1,-0.030284


In [27]:
df_plot = df.copy()
df_plot = df_plot.dropna(subset=["ret"])

df_plot["buy_hold"] = df['ret'].cumsum()
fig = go.Figure()

weekday_names = {
    0: "Mon",
    1: "Tue",
    2: "Wed",
    3: "Thu",
    4: "Fri"
}

for wd in range(5):
    ret_wd = np.where(df_plot["weekday"] == wd, df_plot["ret"], 0)
    cum_wd = ret_wd.cumsum()

    fig.add_trace(
        go.Scatter(
            x=df_plot.index,
            y=cum_wd,
            name=weekday_names[wd],
            line=dict(width=2)
        )
    )



fig.update_layout(
    title="Retorno acumulado por dia da semana",
    xaxis_title="Date",
    yaxis_title="Índice (Base 100)",
    template="plotly_white",
    height=600, width=800
)

fig.show()



In [28]:
# =========================
# DIAS DA SEMANA
# =========================
weekdays_selected = [0, 1, 2, 3,]

df_plot = df.copy()
df_plot = df_plot.dropna(subset=["ret"])

# Retorno filtrado por weekday
df_plot["ret_filtered"] = np.where(
    df_plot["weekday"].isin(weekdays_selected),
    df_plot["ret"],
    0
)

# Retorno simples acumulado (cumsum)
df_plot["cum_filtered"] = df_plot["ret_filtered"].cumsum()
df_plot["cum_bh"] = df_plot["ret"].cumsum()

fig = go.Figure()

# Estratégia filtrada
fig.add_trace(
    go.Scatter(
        x=df_plot.index,
        y=df_plot["cum_filtered"] * 100,
        mode="lines",
        name=f"Weekdays {weekdays_selected}",
        line=dict(width=2)
    )
)

# Buy & Hold
fig.add_trace(
    go.Scatter(
        x=df_plot.index,
        y=df_plot["cum_bh"] * 100,
        mode="lines",
        name="Buy & Hold",
        line=dict(width=2)
    )
)

fig.update_layout(
    title=f"Retorno simples acumulado (cumsum) | Weekdays {weekdays_selected} vs Buy & Hold",
    xaxis_title="Date",
    yaxis_title="Retorno acumulado simples (%)",
    template="plotly_white",
    height=600,
    width=800
)

fig.show()

In [29]:
# =========================
# RETORNO TOTAL (SIMPLES)
# =========================
total_return_bh = df_plot["cum_bh"].iloc[-1]
total_return_weekday = df_plot["cum_filtered"].iloc[-1]

# =========================
# VOLATILIDADE ANUALIZADA
# =========================
vol_bh = df_plot["ret"].std() * np.sqrt(252)
vol_weekday = df_plot["ret_filtered"].std() * np.sqrt(252)

# =========================
# SHARPE (retorno simples)
# =========================
sharpe_bh = (
    df_plot["ret"].mean() / df_plot["ret"].std()
) * np.sqrt(252)

sharpe_weekday = (
    df_plot["ret_filtered"].mean() / df_plot["ret_filtered"].std()
) * np.sqrt(252)

print("=== RESULTADOS | RETORNO SIMPLES (cumsum) ===")
print(f"Buy & Hold Return: {total_return_bh:.2%}")
print(f"Weekdays Return:   {total_return_weekday:.2%}")
print()
print(f"Buy & Hold Vol: {vol_bh:.2%}")
print(f"Weekdays Vol:   {vol_weekday:.2%}")
print()
print(f"Buy & Hold Sharpe: {sharpe_bh:.2f}")
print(f"Weekdays Sharpe:   {sharpe_weekday:.2f}")


=== RESULTADOS | RETORNO SIMPLES (cumsum) ===
Buy & Hold Return: 549.29%
Weekdays Return:   581.95%

Buy & Hold Vol: 41.22%
Weekdays Vol:   37.31%

Buy & Hold Sharpe: 0.52
Weekdays Sharpe:   0.60
